In [1]:
from pathlib import Path
import yaml
import math
import base64

import requests
from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

/home/jviehweger/mambaforge/envs/sh/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def split_frontmatter_content(content):
    fm_split = content.split("---\n", 3)
    front_matter = yaml.safe_load(fm_split[1])
    content = fm_split[-1]
    return front_matter, content

In [41]:
dataset_lookup = {
    "S2L1C": DataCollection.SENTINEL2_L1C,
    "S2L2A": DataCollection.SENTINEL2_L2A,
}

configs = {
    "CDSE": SHConfig("cdse-jonas"),
    "EOB": SHConfig()  
}

In [42]:
readme_paths = list(Path("../sentinel-2/").rglob("*.md"))

In [56]:
def create_example_images(readme_path):
    with open(readme_path, 'r') as original: 
        fm, _ = split_frontmatter_content(original.read())
    for i, example in enumerate(fm.get("examples", [])):
        example_path = readme_path.parent / "fig" / f"example-{i}.jpg"
        if example_path.exists():
            continue
        print(readme_path)
        # Get evalscript
        url = example.get("evalscripturl")
        if url is None:
            evalscript = base64.b64decode(example["evalscript"]).decode('utf-8')
        else:
            evalscript = requests.get(url.replace("eob.js", "script.js")).text

        lat = float(example["lat"])
        lon = float(example["lng"])

        # m per pixel for zoom and centroid
        m_per_pixel = math.floor(40075016.686 * abs(math.cos(lat / 180 * math.pi)) / math.pow(2, int(example["zoom"])+8))
        bbox = (BBox(
            ((lon-0.001, lat-0.001), (lon, lat)), 
            crs=CRS(4326))
            .transform(CRS(3857))
            .buffer((m_per_pixel*1000, m_per_pixel*500), relative=False))
        
        config = configs["EOB"]

        # Get data
        request = SentinelHubRequest(
            evalscript=evalscript,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=dataset_lookup[example["datasetId"]],
                    time_interval=(example["fromTime"], example["toTime"]),
                )
            ],
            responses=[SentinelHubRequest.output_response("default", MimeType.JPG)],
            bbox=bbox,
            resolution=(m_per_pixel, m_per_pixel),
            config=config,
            data_folder="."
        )
        _ = request.get_data(save_data=True)

        # Move data
        output = Path(request.get_filename_list()[0])
        example_path.parent.mkdir(parents=True, exist_ok=True)
        output.rename(example_path)
        (output.parent / "request.json").unlink() 
        output.parent.rmdir()


In [64]:
for readme_path in readme_paths:
    create_example_images(readme_path)

../sentinel-2/sentinel-2.md
../sentinel-2/lai/README.md
../sentinel-2/monthly_snow_report/README.md
../sentinel-2/ulyssys_water_quality_viewer/README.md
../sentinel-2/ccc/README.md
../sentinel-2/oil-spill-index/README.md
../sentinel-2/ozone_column_over_snow/README.md
../sentinel-2/evi2/README.md
../sentinel-2/burned_area_ms/README.md
../sentinel-2/built_up_classifier_mask/README.md
../sentinel-2/aesthetic-neon/README.md
../sentinel-2/bare_soil_detector/README.md
../sentinel-2/ndii/README.md
../sentinel-2/l1c_optimized/README.md
../sentinel-2/hollstein/README.md
../sentinel-2/mixlai/README.md
../sentinel-2/tonemapped_natural_color/README.md
../sentinel-2/evi/README.md
../sentinel-2/indexdb/README.md
../sentinel-2/false-color-urban-rgb/README.md
../sentinel-2/land_use_with_linear_discriminant_analysis/README.md
../sentinel-2/deep_bais2/README.md
../sentinel-2/forest_cut_temporal_detection/README.md
../sentinel-2/psri/README.md
../sentinel-2/simple_water_bodies_mapping-swbm/README.md
../s

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
400 Client Error: Bad Request for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 400, "reason": "Bad Request", "message": "Failed to evaluate script!\nundefined:2: SyntaxError: Unexpected token :\n404: Not Found\n   ^\n", "code": "RENDERER_EXCEPTION"}"